In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## gensim word2vec preparation

In [3]:
!pip install -U gensim
#we use this for the word2vec representations of each word
#remove if not using google colab. Instead, install from console

    100% |████████████████████████████████| 22.6MB 2.1MB/s 
Requirement not upgraded as not directly required: scipy>=0.18.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (0.19.1)
Requirement not upgraded as not directly required: six>=1.5.0 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: numpy>=1.11.3 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.14.5)
    100% |████████████████████████████████| 1.4MB 16.6MB/s 
Requirement not upgraded as not directly required: requests in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (2.18.4)
    100% |████████████████████████████████| 133kB 20.6MB/s 
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /usr/local/lib/python3.6/dist-packages (from requests->smart-open>=1.2.1->gensim) (3.0.4)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from re

use gensim to get the word2vec representations of each word  
reference here:  
https://www.quora.com/How-do-you-extract-vectors-from-word2vec-given-a-word

In [0]:
from gensim.models import KeyedVectors#, Word2Vec

#currently using gensim's default word2vec pretrained model, trained on googlenews data. 
#todo: train on wikiqa if the results are crap. 

w2vmodel = KeyedVectors.load_word2vec_format('drive/Colab Notebooks/datasets/word2vec/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

word2vec first testing

In [5]:

# print(w2vmodel.similarity('woman','man'))
wvdict = w2vmodel.wv.vocab
#this is the list of all the words in the vocabulary

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [40]:
print(w2vmodel.index2word[0])

in


## Preprocessing input strings

In [0]:
import numpy as np

`a.toks` contains the questions (repeat question lines correspond to the number of candidate answer sentences)  
`b.toks` contains the answers. Each line is the answer to the question on the same line of `a.toks`  
`id` contains the id of the question at that line in `a.toks`  
`sim` contains the labels: for each block of repeated-questions: there is one line containing a `1` corresponding to the line in `b.toks` containing the correct answer to the same line in `a.toks`

https://github.com/castorini/data/tree/master/WikiQA

In [0]:
def read_data(path, readtype):
  with open(path) as f:
    lines = f.readlines()
    
  if readtype == "questions" or readtype == "answers":
    return lines
  elif readtype == "labels":
    return [int(line) for line in lines]

Load the data

In [0]:
bigpath = "drive/Colab Notebooks/datasets/QuestionAnswering/TrecQA/"
questions = read_data(bigpath+"/train-all/a.toks", "questions")
answers = read_data(bigpath+"/train-all/b.toks", "answers")
labels = read_data(bigpath+"/train-all/sim.txt", "labels")
#question_ids = read_data(bigpath+"/train/id.txt", "labels")
#questions

#### preparing word-level sequence embedding

This is to prepare for the Embedding layer

1. concatenate questions and answers into a list of `[question | answer]` pairs. This is a list of sequences.  
2. find max sequence length from this list  
3. the vocab size of the google news word2vec is 3 million words, the number of features is 300

The embedding_matrix:
1. let each word in `w2vmodel.wv.vocab.keys()` have an index
2. concatenate axis=1 all the word vectors in the vocabulary into embedding_matrix
3. then turn each string sequence into an array of numbers (each number represents the index of that word in the vocab)
4. pass the string sequences into the .fit as training input. The shape of this input should be (n_seqs, max_seq_length, n_features)
5. the embedding matrix will be the weights in the Embedding layer

In [0]:
vocab_size = 299567+1 #this is the SLIMmed googlenews w2v model's vocab count. #the last item is the 'all-zeroes' word vector, used for words that aren't in the w2vmodel

In [10]:
#make embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))
for i in range(len(wvdict)):
    embedding_vector = w2vmodel.wv[w2vmodel.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
embedding_matrix[299566]

#### define some functions to preprocess data (both train and test)

In [0]:
def SEQ_max_length(sequences):
  return max( [len(qa.split()) for qa in sequences] )

In [12]:
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
def SEQ_token_and_pad(sequences, max_seq_length):
  
#   #version 0a
#   t = Tokenizer()
#   t.fit_on_texts(sequences)

#   vocab_size = len(t.word_index)+1

#   encoded_seqs = t.texts_to_sequences(sequences)
#   print(encoded_seqs)

#   #version 0b and up
  encoded_seqs = []
  for qa in sequences:
    encoded_qa = []
    tokenized_qa = qa.split()
    for word in tokenized_qa:
      index_to_append = vocab_size - 1
      
      if word in wvdict.keys():
        index_to_append = w2vmodel.vocab[word].index
      encoded_qa.append(index_to_append)
    
    encoded_seqs.append(encoded_qa)
    
  #print(encoded_seqs)
  
  padded_seqs = pad_sequences(encoded_seqs, maxlen = max_seq_length, padding="post", value = vocab_size - 1)
  return padded_seqs

#version 0
#padded_seqs = SEQ_token_and_pad(sequences, max_seq_length)


Performing the necessary preprocessing on our data using our functions

In [0]:

max_question_length = SEQ_max_length(questions)
max_answer_length = SEQ_max_length(answers)
max_hidden_length = max(max_question_length, max_answer_length)

#version 1: getting separate questions and answers vectors (but still encoded in index form, not in pure word vector form)
padded_questions = SEQ_token_and_pad(questions, max_hidden_length)
padded_answers = SEQ_token_and_pad(answers, max_hidden_length)


In [35]:
#rebuild a question and answer from the padded representation
print(padded_questions[0])
print(max_hidden_length)

" ".join([w2vmodel.wv.index2word[index] for index in padded_answers[0] if not index == 299567])



[   132     17  27136  22292   2874 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567]
40


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


'partly submerged glacier cave on moreno glacier'

#### creating one-hot labels

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
labels_np = np.array(labels)
labels_np = np.expand_dims(labels_np, axis=1)

onehot = OneHotEncoder(n_values = 2)
onehot.fit(labels_np)
labels_np = onehot.transform(labels_np)

## Model building and training

### Guesswork and experimentation



---


*If i'm doing the implementation in the 1506.06490 paper*  
**Todo**: 

- find length of the longest question and longest answer
- use these maxes for padding to shorter sentences
- in one big model, build 2 separate convnet submodels, one for questions, one for answers
- concatenate the outputs for the questions and answers convnets, and we obtain the **joint representation** `p` of each question-answer pair.

Each of these learned joint pairs is a single **timestep** in the LSTM part. We learn the sequence and semantic relationships between each answer-candidate sentence in the context paragraph, not between the words themselves. (the word-level patterns are learned in the Conv stage already)

for the LSTM stage, padding instead involves adding dummy qa pairs for each timestep without an answer. The max number of answer candidates in the dataset (and possible qa pairs, per question) is 30, so any Qs with fewer than 30 answer sentences will be padded with dummy pairs.


---



*my own simple-ass idea*  
**Todo:**

version 0a: `shit lstm model copped from some blog`  
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

version 0b: `with learning rate, and fixed word tokenizations`

version 1a: `separate LSTM for question and answer, then Concatenate into a hidden layer which then is fed into classifier/Dense/whatever`

version 1b: `no LSTM for question and answer. Using Functional API. WORKS but OVERFITS.`

### Giang's idea

Giang's idea:

`doc question  
input_1 = Input(...)  
...  
output_1 = ...


doc answer  
input_2 = Input(...)  
...  
output_2 = ...

input_2 = Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_answer_length, trainable=False)  
  
output_2 = LSTM(100)(input_reshape_2)

hidden = Concatenate()([output_1, output_2])
classifier = Dense(1, activation="sigmoid")(hidden)
`

In [0]:
# #not using functional API
# #dont use this, non working

# questions_model = Sequential()
# questions_model.add(equestions)
# #questions_model.add(LSTM(64))
# # questions_model.add(Activation('softmax'))
# #questions_model.add(Dropout(0.4))

# answers_model = Sequential()
# answers_model.add(eanswers)
# #answers_model.add(LSTM(64))
# # answers_model.add(Activation('softmax'))
# #answers_model.add(Dropout(0.4))

# merged = Merge([questions_model.output, answers_model.output], mode = "concat")
# print(merged.output.shape)
# final_model = Sequential()
# final_model.add(merged)
# #final_model.add(Flatten())
# final_model.add(LSTM(100)) #comment out if using lstm on the question and answer submodels
# final_model.add(Dense(2, activation="softmax"))

# final_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])
# print(final_model.summary())

In [17]:
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Dropout
from keras.optimizers import Adam

#using keras functional api

qumodel_input = Input(shape = (max_hidden_length,), name="questionsMainInput")
qumodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(qumodel_input)
qumodel_parse = Conv1D(32, 2)(qumodel_embed)
qumodel_parse = Dropout(0.2)(qumodel_parse)
qumodel_parse = MaxPooling1D(pool_size = 2)(qumodel_parse)

ansmodel_input = Input(shape = (max_hidden_length,), name="answersMainInput")
ansmodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(ansmodel_input)
ansmodel_parse = Conv1D(32,2)(ansmodel_embed)
ansmodel_parse = Dropout(0.2)(ansmodel_parse)
ansmodel_parse = MaxPooling1D(pool_size = 2)(ansmodel_parse)

combinedmodel = Concatenate()([qumodel_parse, ansmodel_parse])
print(combinedmodel.shape)

lstmcombinedmodel = LSTM(64)(combinedmodel)
classifier = Dense(2, activation='softmax')(lstmcombinedmodel)

final_model = Model(inputs = [qumodel_input, ansmodel_input], outputs = classifier)

final_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr = 0.003), metrics=['accuracy'])
print(final_model.summary())

(?, 29, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
questionsMainInput (InputLayer) (None, 60)           0                                            
__________________________________________________________________________________________________
answersMainInput (InputLayer)   (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 300)      89870400    questionsMainInput[0][0]         
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 300)      89870400    answersMainInput[0][0]           
_________________________________________________________________________________________________

### Attempting the 1506 06490 paper

In [22]:
from keras.layers import Dense, LSTM, Conv1D, Embedding, MaxPooling1D, Activation, Concatenate
from keras.models import Model
from keras.optimizers import Adadelta

In [25]:
#embedding is Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False)

questions_model = Sequential()
questions_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
questions_model.add(Conv1D(100, 5, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=1))
questions_model.add(Activation('relu'))

answers_model = Sequential()
answers_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
answers_model.add(Conv1D(100, 5, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=1))
answers_model.add(Activation('relu'))

merge = Merge([questions_model, answers_model])

print(merge.output.shape)

final_model = Sequential()
final_model.add(merge)
final_model.add(Flatten())
# final_model.add(LSTM(160))
final_model.add(Dense(1, activation="softmax"))

final_model.compile(loss = 'binary_crossentropy', optimizer = Adadelta(lr=0.5), metrics=['accuracy'])
print(final_model.summary())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


(?, 6, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 6, 100)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 601       
Total params: 180,162,001
Trainable params: 421,201
Non-trainable params: 179,740,800
_________________________________________________________________
None


In [36]:
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.optimizers import Adadelta
from keras.regularizers import l1

#using keras functional api
cnns = [5, 2, 2]
pools = [2, 2, 1]

qumodel_input = Input(shape = (max_hidden_length,), name="questionsMainInput")
qumodel_parse = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(qumodel_input)
for layer in range(len(cnns)):
  qumodel_parse = Conv1D(100, cnns[layer])(qumodel_parse)
  qumodel_parse = Dropout(0.2)(qumodel_parse)
  qumodel_parse = MaxPooling1D(pool_size = pools[layer])(qumodel_parse)
  qumodel_parse = Activation('relu')(qumodel_parse)

print(qumodel_parse.shape)

ansmodel_input = Input(shape = (max_hidden_length,), name="answersMainInput")
ansmodel_parse = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(ansmodel_input)
for layer in range(len(cnns)):
  ansmodel_parse = Conv1D(100, cnns[layer])(ansmodel_parse)
  ansmodel_parse = Dropout(0.2)(ansmodel_parse)
  ansmodel_parse = MaxPooling1D(pool_size = pools[layer])(ansmodel_parse)
  ansmodel_parse = Activation('relu')(ansmodel_parse)
  
print(ansmodel_parse.shape)

combinedmodel = Concatenate()([qumodel_parse, ansmodel_parse])
print(combinedmodel.shape)

lstmcombinedmodel = LSTM(100)(combinedmodel)
classifier = Dense(2, activation='softmax')(lstmcombinedmodel)

final_model = Model(inputs = [qumodel_input, ansmodel_input], outputs = classifier)

final_model.compile(loss = 'categorical_crossentropy', optimizer = Adadelta(lr = 0.01), metrics=['accuracy'])
print(final_model.summary())

(?, 7, 100)
(?, 7, 100)
(?, 7, 200)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
questionsMainInput (InputLayer) (None, 40)           0                                            
__________________________________________________________________________________________________
answersMainInput (InputLayer)   (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 40, 300)      89870400    questionsMainInput[0][0]         
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 40, 300)      89870400    answersMainInput[0][0]           
_________________________________________________________________________

### Attempting the dl-models-for-qa QA-LSTM model

In [46]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Merge, Dense
WORD2VEC_EMBED_SIZE = 300
QA_EMBED_SIZE = 64
BATCH_SIZE = 64
NBR_EPOCHS = 10

qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_matrix], mask_zero=True))
qenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, input_shape=(None, 300, max_hidden_length), 
                            return_sequences=False), merge_mode="sum"))
qenc.add(Dropout(0.3))

aenc = Sequential()
aenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_matrix], mask_zero=True))
aenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, input_shape=(None, 300, max_hidden_length), 
                            return_sequences=False), merge_mode="sum"))
aenc.add(Dropout(0.3))

model = Sequential()
model.add(Merge([qenc, aenc], mode="sum"))
model.add(Dense(1, activation="softmax"))

model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=["accuracy"])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 180,114,625
Trainable params: 180,114,625
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


## Fit the chosen model

Severely punish "no" answers because that's just lazy. The network should at least try to guess "yes" or "1" on at least some questions, and not settle for a 88% accuracy by answering no to everything.

add class_weights and try:

In [17]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(labels), labels)
#class_weights = np.array([0.5, 24.])

In [18]:
#final_model.fit([padded_questions, padded_answers], labels, epochs=5, batch_size=128, verbose=1, class_weight = class_weights, validation_split=0.2)

#model.fit([padded_questions, padded_answers], labels, batch_size=32,
 #         epochs=20, validation_split=0.1, class_weight = class_weights)
  
final_model.fit([padded_questions, padded_answers], labels_np, epochs=30, batch_size=64, verbose=1, validation_split=0.1)

Train on 48075 samples, validate on 5342 samples
Epoch 1/30
44352/48075 [==========================>...] - ETA: 4s - loss: 0.3436 - acc: 0.8782

48075/48075 [==============================] - 64s 1ms/step - loss: 0.3410 - acc: 0.8787 - val_loss: 0.3463 - val_acc: 0.8843
Epoch 2/30
27456/48075 [================>.............] - ETA: 25s - loss: 0.3006 - acc: 0.8839

48075/48075 [==============================] - 62s 1ms/step - loss: 0.2962 - acc: 0.8852 - val_loss: 0.3633 - val_acc: 0.8811
Epoch 3/30
21056/48075 [============>.................] - ETA: 34s - loss: 0.2716 - acc: 0.8921

48075/48075 [==============================] - 62s 1ms/step - loss: 0.2674 - acc: 0.8946 - val_loss: 0.3501 - val_acc: 0.8778
Epoch 4/30
18624/48075 [==========>...................] - ETA: 37s - loss: 0.2488 - acc: 0.9013

48075/48075 [==============================] - 62s 1ms/step - loss: 0.2476 - acc: 0.9023 - val_loss: 0.3492 - val_acc: 0.8759
Epoch 5/30
 5056/48075 [==>...........................] - ETA: 52s - loss: 0.2377 - acc: 0.9029

KeyboardInterrupt: ignored

## Testing the trained model

#### Preparing the test data

In [0]:
test_questions = ["Where was Tuan born","Where was Tuan born","Where was Tuan born","Where was Tuan born"]
test_answers = ["Pigs dogs cats","Boo","Candy balls cake", "Tuan was born in Vietnam"]
test_labels = [0, 0, 0, 1]



# test_questions = read_data(bigpath+"/test/a.toks", "questions")
# test_answers = read_data(bigpath+"/test/b.toks", "answers")
# test_labels = read_data(bigpath+"/test/sim.txt", "labels")
# question_ids = read_data(bigpath+"/dev/id.txt", "labels")

In [0]:
#test_seqs, max_test_seq_length = SEQ_make_qa_sequences_list(test_questions, test_answers)
padded_test_questions = SEQ_token_and_pad(test_questions, max_hidden_length)
padded_test_answers = SEQ_token_and_pad(test_answers, max_hidden_length)

#### Predicting some q, a pairs

In [0]:
test_preds = final_model.predict_on_batch([padded_test_questions, padded_test_answers])

In [26]:
print(test_preds)
np.argmax(test_preds*1000)

[[9.9997854e-01 2.1408780e-05]
 [9.9997807e-01 2.1898602e-05]
 [9.9996340e-01 3.6599067e-05]
 [9.9997723e-01 2.2740214e-05]]


0

In [22]:
i=4 #test example to check

# print(padded_test_seqs[1])
# print(padded_test_seqs[16])

print(test_preds[i])
print(test_questions[i])
print(test_answers[i])
print("correct label: "+str(test_labels[i]))

[9.9986219e-01 1.3773878e-04]
how are glacier caves formed ?

glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice .

correct label: 0
